#Calculate Average for movie ratings


In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 54.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=2c5017bb405f72285b5e3daefc86968fceec4c2ff5549cd3e9bbf2061721f222
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [4]:
from pyspark import SparkConf, SparkContext
Conf = SparkConf().setAppName("Finding Average for movie ratings")
sc = SparkContext.getOrCreate(conf = Conf)

In [91]:
def addition(listV):
  sum = 0
  i = 0
  for i in range(0,len(listV)):
    sum = sum+int(listV[i]) 
  return sum/(i+1)

In [92]:
csv_File = sc.textFile("/content/sample_data/movie_ratings.csv")
csv_File.map(lambda x : tuple(x.split(","))).groupByKey().mapValues(list).map(lambda x : (x[0],addition(x[1]))).collect()

[('The Shawshank Redemption', 3.0),
 ('12 Angry Men', 2.2),
 ('The Godfather', 4.333333333333333),
 ('The Matrix', 4.666666666666667),
 ('Pulp Fiction', 3.8)]

# Using Reduce By Key : 

In [94]:
csv_File_1 = sc.textFile("/content/sample_data/movie_ratings.csv")
csv_File_1.collect()

['The Shawshank Redemption,3',
 'The Matrix,5',
 '12 Angry Men,3',
 '12 Angry Men,4',
 'The Matrix,5',
 'Pulp Fiction,4',
 'The Godfather,5',
 'The Shawshank Redemption,2',
 'Pulp Fiction,4',
 'The Godfather,5',
 '12 Angry Men,2',
 'The Godfather,3',
 'Pulp Fiction,4',
 '12 Angry Men,1',
 'The Shawshank Redemption,2',
 '12 Angry Men,1',
 'The Shawshank Redemption,5',
 'Pulp Fiction,5',
 'Pulp Fiction,2',
 'The Matrix,4']

In [127]:
csv_File_1 = csv_File_1.repartition(10)

csv_File_1.getNumPartitions()

10

In [129]:
csv_File_2 = csv_File_1.map(lambda x : (x.split(",")[0], (int(x.split(",")[1]),1))).reduceByKey(lambda x,y : (x[0]+y[0],x[1]+y[1])).map(lambda x: (x[0], x[1][0]/x[1][1]))
csv_File_2.collect()

[('The Shawshank Redemption', 3.0),
 ('12 Angry Men', 2.2),
 ('The Matrix', 4.666666666666667),
 ('The Godfather', 4.333333333333333),
 ('Pulp Fiction', 3.8)]

In [130]:
csv_File_2.saveAsTextFile("/content/sample_data/movie_ratings")